<a href="https://colab.research.google.com/github/Sainzrt/CALCULADORA-CREDITICIA/blob/main/FINAL_SIMULCREDIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Pega TODO esto en una sola celda en Google Colab y ejecútalo.
from IPython.display import display, HTML, Javascript
import pandas as pd
import io, re, os, time, json, base64
from google.colab import output, files
import matplotlib.pyplot as plt
import numpy as np
import requests

# ---------------------------
# === ESTILO (idéntico) ===
# ---------------------------
style = """
<style>
body {
  background-color: #f8fbff;
  color: #0a2540;
  font-family: 'Poppins', sans-serif;
}

.simucalc-container {
  background-color: white;
  border-radius: 18px;
  box-shadow: 0 10px 30px rgba(0, 64, 128, 0.1);
  width: 520px;
  margin: 70px auto;
  padding: 45px 50px;
  text-align: center;
  border: 1px solid #e1ebf5;
}

h1 {
  color: #12355b;
  font-weight: 700;
  font-size: 2.1rem;
  letter-spacing: 1px;
  margin-bottom: 35px;
}

label {
  display: block;
  font-size: 0.95rem;
  margin: 18px 0 6px;
  color: #1c3d6d;
  text-align: left;
  font-weight: 500;
}

input {
  width: 100%;
  padding: 12px;
  border-radius: 10px;
  border: 1px solid #a7c4e0;
  outline: none;
  font-size: 1rem;
  background-color: #f3f8fc;
  color: #0a2540;
  transition: 0.3s;
}

input:focus {
  border-color: #3075b5;
  background-color: #eef6ff;
  box-shadow: 0 0 6px rgba(0, 102, 204, 0.2);
}

button {
  background-color: #164b88;
  border: none;
  color: white;
  padding: 13px 28px;
  border-radius: 10px;
  font-size: 1rem;
  cursor: pointer;
  margin-top: 30px;
  font-weight: 600;
  letter-spacing: 0.3px;
  transition: 0.3s ease-in-out;
  width: 100%;
}

button:hover {
  background-color: #1f5aa8;
  transform: translateY(-2px);
  box-shadow: 0 4px 15px rgba(22, 75, 136, 0.3);
}

footer {
  text-align: center;
  color: #5d7da1;
  margin-top: 30px;
  font-size: 0.9rem;
  letter-spacing: 0.4px;
}

.brand-line {
  width: 70px;
  height: 3px;
  background-color: #164b88;
  margin: -15px auto 30px;
  border-radius: 3px;
}

#resultado {
  width: 520px;
  margin: 24px auto 60px;
  background: white;
  padding: 22px;
  border-radius: 12px;
  box-shadow: 0 5px 20px rgba(0,0,0,0.03);
  border: 1px solid #e1ebf5;
  text-align: left;
}

.table-styles {
  width: 100%;
  border-collapse: collapse;
  margin-top: 10px;
  font-size: 0.95rem;
  color: #0a2540;
}

.table-styles th, .table-styles td {
  padding: 10px 8px;
  border-bottom: 1px solid #eef4fb;
  text-align: right;
}

.table-styles th {
  text-align: left;
  color: #12355b;
  font-weight: 600;
  background: transparent;
}

.download-btn {
  display:inline-block;
  margin-top:15px;
  background-color:#164b88;
  color:white;
  padding:10px 18px;
  border-radius:8px;
  text-decoration:none;
  font-weight:600;
  cursor:pointer;
  transition:0.3s;
}

.download-btn:hover {
  background-color:#1f5aa8;
  transform:translateY(-2px);
}

/* Finanbot widget en esquina inferior derecha (no modifica tu formato principal) */
#finanbot_widget {
  position: fixed;
  right: 18px;
  bottom: 18px;
  width: 340px;
  max-width: 92%;
  background: linear-gradient(180deg, #ffffff, #f1f7ff);
  border-radius: 12px;
  box-shadow: 0 10px 30px rgba(0,64,128,0.12);
  border: 1px solid #e6f0ff;
  z-index: 9999;
  overflow: hidden;
  display: none; /* aparece solo en segunda página */
}

#finanbot_header {
  background: linear-gradient(90deg,#164b88,#1f5aa8);
  color:white;
  padding:10px 12px;
  font-weight:700;
  display:flex;
  justify-content:space-between;
  align-items:center;
}
#finanbot_messages { height:220px; overflow:auto; padding:10px; background:white;}
#finanbot_input { border-top:1px solid #eef4fb; display:flex; gap:8px; padding:8px; background:#fbfdff;}
#finanbot_input input { flex:1; padding:8px 10px; border-radius:8px; border:1px solid #dfeefc; outline:none;}
#finanbot_input button { padding:8px 10px; border-radius:8px; background:#164b88; color:white; border:none; cursor:pointer; font-weight:600;}

/* pequeño badge sugerente */
#finanbot_toggle { position: fixed; right: 22px; bottom: 18px; width:56px; height:56px; border-radius:50%; background:#164b88; color:white; display:flex; align-items:center; justify-content:center; font-weight:700; z-index:9998; cursor:pointer; box-shadow: 0 8px 22px rgba(22,75,136,0.25);}
</style>
"""

# ---------------------------
# === HTML (PIDO: no tocar formato principal) ===
# ---------------------------
html = """
<div class="simucalc-container">
  <h1>SIMULCREDIT</h1>
  <div class="brand-line"></div>

  <label for="monto">Monto del crédito (USD):</label>
  <input type="number" id="monto" placeholder="Ej. 15000" />

  <label for="tasa">Tasa de interés anual (%):</label>
  <input type="number" id="tasa" placeholder="Ej. 7.5" />

  <label for="plazo">Plazo (meses):</label>
  <input type="number" id="plazo" placeholder="Ej. 36" />

  <button onclick="google.colab.kernel.invokeFunction('calc_credit_backend',
    [parseFloat(document.getElementById('monto').value) || 0,
     parseFloat(document.getElementById('tasa').value) || 0,
     parseInt(document.getElementById('plazo').value) || 0], {})">
     Calcular crédito
  </button>

  <div style="margin-top:12px;">
    <button class='download-btn' onclick="google.colab.kernel.invokeFunction('abrir_tasas_inicial', [], {})">🏦 Integrar tasas internacionales</button>
    <button class='download-btn' onclick="google.colab.kernel.invokeFunction('abrir_componente_educativo_inicial', [], {})">📚 Componente educativo</button>
  </div>
</div>

<div id="resultado"></div>

<!-- Finanbot widget (inicialmente hidden; se mostrará sólo en segunda página) -->
<div id="finanbot_widget">
  <div id="finanbot_header">
    <div>🤖 Finanbot</div>
    <div style="font-size:0.9rem; opacity:0.9;"><span id="finanbot_status">offline</span></div>
  </div>
  <div id="finanbot_messages"></div>
  <div id="finanbot_input">
    <input id="finanbot_text" placeholder="Pregunta sobre tu crédito, pagos extra, plazo, tasas..." />
    <button onclick="enviarPreguntaFinanbot()">Enviar</button>
  </div>
</div>

<div id="finanbot_toggle" title="Abrir Finanbot" style="display:none;">💬</div>

<footer>© 2025 Simulcredit | Plataforma de simulación financiera</footer>
"""

display(HTML(style + html))

# ---------------------------
# === FUNCIONES FINANCIERAS ===
# ---------------------------
def generar_tabla_amortizacion(monto, tasa_anual, plazo_meses):
    if plazo_meses <= 0:
        return pd.DataFrame([], columns=["Mes", "Pago total", "Capital", "Interés", "Saldo restante"])
    if tasa_anual <= 0:
        cuota = monto / plazo_meses
        saldo = monto
        datos = []
        for i in range(1, plazo_meses + 1):
            interes = 0.0
            capital = cuota
            saldo -= capital
            datos.append([i, round(cuota,2), round(capital,2), round(interes,2), round(max(saldo,0),2)])
        return pd.DataFrame(datos, columns=["Mes", "Pago total", "Capital", "Interés", "Saldo restante"])
    tasa_mensual = tasa_anual / 12 / 100
    cuota = monto * (tasa_mensual * (1 + tasa_mensual)**plazo_meses) / ((1 + tasa_mensual)**plazo_meses - 1)
    saldo = monto
    datos = []
    for i in range(1, plazo_meses + 1):
        interes = saldo * tasa_mensual
        capital = cuota - interes
        saldo -= capital
        datos.append([i, round(cuota, 2), round(capital, 2), round(interes, 2), round(max(saldo,0), 2)])
    return pd.DataFrame(datos, columns=["Mes", "Pago total", "Capital", "Interés", "Saldo restante"])

# Estado global (segunda página)
ultima_tasa = None
ultimo_plazo = None
ultimo_monto = None
ultima_df = None

# ---------------------------
# === CALLBACK: calcular y mostrar segunda página ===
# ---------------------------
def calc_credit_backend(monto, tasa, plazo):
    global ultima_tasa, ultimo_plazo, ultimo_monto, ultima_df
    try:
        monto = float(monto)
        tasa = float(tasa)
        plazo = int(plazo)
    except:
        display(Javascript("""document.getElementById('resultado').innerHTML =
            '<b style="color:#b30000;">Por favor ingresa valores válidos.</b>'"""))
        return
    if monto <= 0 or plazo <= 0 or tasa < 0:
        display(Javascript("""document.getElementById('resultado').innerHTML =
            '<b style="color:#b30000;">Por favor ingresa valores válidos (monto>0, plazo>0, tasa≥0).</b>'"""))
        return

    df = generar_tabla_amortizacion(monto, tasa, plazo)
    ultima_df = df.copy()
    ultima_tasa, ultimo_plazo, ultimo_monto = tasa, plazo, monto

    # Guardar Excel (mismo comportamiento)
    nombre_archivo = "Tabla_Amortizacion_Simulcredit.xlsx"
    excel_buffer = io.BytesIO()
    df.to_excel(excel_buffer, index=False, engine='openpyxl')
    excel_buffer.seek(0)
    with open(nombre_archivo, "wb") as f:
        f.write(excel_buffer.read())

    # Resumen crediticio
    cuota = df['Pago total'].iloc[0] if not df.empty else 0
    total_pagado = df['Pago total'].sum()
    total_intereses = df['Interés'].sum()
    porcentaje_intereses = (total_intereses / total_pagado * 100) if total_pagado>0 else 0

    resumen_html = f"""
      <div style="background:#eef6ff; padding:15px; border-radius:10px; margin-bottom:15px;">
        <b>Resumen crediticio:</b><br>
        Cuota mensual: <b>${cuota:,.2f}</b><br>
        Total pagado: <b>${total_pagado:,.2f}</b><br>
        Intereses totales: <b>${total_intereses:,.2f}</b>
      </div>
    """

    # Tabla EXACTA (sin tocar formato)
    def fmt(x): return f"{x:,.2f}"
    html_tabla = """
    <h3 style="color:#12355b; margin-top:0;">Tabla de Amortización</h3>
    <table class="table-styles">
      <thead>
        <tr>
          <th>Mes</th><th>Pago total (USD)</th><th>Capital (USD)</th>
          <th>Interés (USD)</th><th>Saldo restante (USD)</th>
        </tr>
      </thead><tbody>
    """
    for _, row in df.iterrows():
        html_tabla += f"""
          <tr>
            <td style='text-align:left;'>{int(row['Mes'])}</td>
            <td>{fmt(row['Pago total'])}</td>
            <td>{fmt(row['Capital'])}</td>
            <td>{fmt(row['Interés'])}</td>
            <td>{fmt(row['Saldo restante'])}</td>
          </tr>"""
    html_tabla += "</tbody></table>"

    # Botones en ORDEN: Dashboard, Finanbot (toggle), Simulador
    botones = """
      <div style="margin-top:12px;">
        <button class='download-btn' onclick="google.colab.kernel.invokeFunction('descargar_excel', [], {})">📥 Descargar Excel</button>
      </div>

      <div style="margin-top:10px;">
        <button class='download-btn' onclick="google.colab.kernel.invokeFunction('abrir_dashboard', [], {})">📊 Dashboard</button>
      </div>

      <div style="margin-top:8px;">
        <button class='download-btn' onclick="google.colab.kernel.invokeFunction('activar_finanbot_widget', [], {})">💬 Finanbot</button>
      </div>

      <div style="margin-top:8px;">
        <button class='download-btn' onclick="google.colab.kernel.invokeFunction('abrir_simulador_modal', [], {})">🔁 Simulador de escenarios</button>
      </div>
    """

    # Append also the earlier buttons for "componente educativo" y "tasas" debajo (opcional)
    botones_inferiores = """
      <div style="margin-top:12px;">
        <button class='download-btn' onclick="google.colab.kernel.invokeFunction('abrir_componente_educativo_segunda', [], {})">📚 Componente educativo</button>
        <button class='download-btn' style="margin-left:10px;" onclick="google.colab.kernel.invokeFunction('abrir_tasas_segunda', [], {})">🏦 Integrar tasas internacionales</button>
      </div>
    """

    display(Javascript(f"""
      document.getElementById('resultado').innerHTML = `{resumen_html + html_tabla + botones + botones_inferiores}`;
      // show Finanbot toggle only in second page (hidden earlier)
      document.getElementById('finanbot_toggle').style.display = 'flex';
    """))

# ---------------------------
# === DESCARGAR EXCEL ===
# ---------------------------
def descargar_excel():
    try:
        files.download("Tabla_Amortizacion_Simulcredit.xlsx")
    except Exception as e:
        display(Javascript(f"document.getElementById('resultado').innerHTML += '<div style=\"color:#b30000; margin-top:8px;\">Error al descargar: {str(e)}</div>';"))

# ---------------------------
# === DASHBOARD (profesional) ===
# ---------------------------
def abrir_dashboard():
    global ultima_df, ultimo_monto, ultimo_plazo, ultimo_tasa
    if ultima_df is None:
        display(Javascript("document.getElementById('resultado').innerHTML += '<div style=\"color:#b30000; margin-top:8px;\">Calcula un crédito primero.</div>';"))
        return
    df = ultima_df.copy()
    cuota = df['Pago total'].iloc[0]
    total_pagado = df['Pago total'].sum()
    total_intereses = df['Interés'].sum()
    total_capital = df['Capital'].sum()

    # Gráficos
    # 1) Saldo vs Mes
    plt.figure(figsize=(5,2.2))
    plt.plot(df['Mes'], df['Saldo restante'], marker='o', linewidth=2)
    plt.title('Saldo restante vs Mes')
    plt.xlabel('Mes')
    plt.ylabel('Saldo restante (USD)')
    plt.tight_layout()
    buf1 = io.BytesIO(); plt.savefig(buf1, format='png', dpi=120, bbox_inches='tight'); plt.close(); buf1.seek(0)
    b641 = base64.b64encode(buf1.read()).decode('utf-8')

    # 2) Pagos acumulados
    cum_cap = np.cumsum(df['Capital'])
    cum_int = np.cumsum(df['Interés'])
    plt.figure(figsize=(5,2.2))
    ind = df['Mes']
    plt.bar(ind, cum_cap, label='Capital acumulado')
    plt.bar(ind, cum_int, bottom=cum_cap, label='Interés acumulado')
    plt.xlabel('Mes'); plt.ylabel('USD acumulado'); plt.legend(); plt.tight_layout()
    buf2 = io.BytesIO(); plt.savefig(buf2, format='png', dpi=120, bbox_inches='tight'); plt.close(); buf2.seek(0)
    b642 = base64.b64encode(buf2.read()).decode('utf-8')

    # 3) Pie
    plt.figure(figsize=(3.5,3.5))
    plt.pie([total_capital, total_intereses], labels=['Capital','Interés'], autopct='%1.1f%%', startangle=90)
    plt.axis('equal')
    buf3 = io.BytesIO(); plt.savefig(buf3, format='png', dpi=120, bbox_inches='tight'); plt.close(); buf3.seek(0)
    b643 = base64.b64encode(buf3.read()).decode('utf-8')

    dash_html = f"""
      <div style="margin-top:12px; padding:12px; border-radius:10px; background:#fff; border:1px solid #e8f1ff;">
        <h3 style="margin-top:0;color:#12355b;">📊 Dashboard profesional</h3>
        <div style="display:flex; gap:10px; flex-wrap:wrap;">
          <div style="flex:1; min-width:240px;">
            <b>KPI</b>
            <div style="margin-top:8px; padding:10px; background:#f7fbff; border-radius:8px;">
              <div><b>Monto:</b> ${ultimo_monto:,.2f}</div>
              <div><b>Plazo:</b> {ultimo_plazo} meses</div>
              <div><b>Cuota mensual:</b> ${cuota:,.2f}</div>
              <div><b>Total pagado:</b> ${total_pagado:,.2f}</div>
              <div><b>Intereses totales:</b> ${total_intereses:,.2f}</div>
            </div>
            <div style="margin-top:8px;"><b>Saldo restante (proyección)</b>
              <div style="margin-top:8px;"><img src="data:image/png;base64,{b641}" style="width:100%; border-radius:8px;" /></div>
            </div>
          </div>

          <div style="width:220px;">
            <b>Composición de pago</b>
            <div style="margin-top:8px;"><img src="data:image/png;base64,{b643}" style="width:100%; border-radius:8px;" /></div>
            <div style="margin-top:8px;"><b>Acumulado</b><div><img src="data:image/png;base64,{b642}" style="width:100%; border-radius:8px;" /></div></div>
          </div>
        </div>
        <p style="color:#4b6b8d; margin-top:10px;"><small>Gráficos generados a partir de la tabla actual.</small></p>
      </div>
    """
    display(Javascript(f"document.getElementById('resultado').innerHTML += `{dash_html}`;"))

# ---------------------------
# === FINANBOT: widget funcional (local heuristics + optional OpenAI) ===
# ---------------------------

# Helper: try to use OpenAI if key present.
USE_OPENAI = False
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY', None)
if OPENAI_API_KEY:
    USE_OPENAI = True

# Local lightweight responder (heuristics & calculations). Keeps scope limited to amortization domain.
def local_finanbot_response(question):
    global ultima_df, ultimo_monto, ultima_tasa, ultimo_plazo
    q = question.lower().strip()

    # If asks about amortization definition
    if re.search(r'\b(amortiz|qué es amortiz|qué es la amortiz|amortización)\b', q):
        return "La amortización es el proceso de pagar una deuda poco a poco mediante cuotas. Cada cuota contiene capital (lo que reduces) y interés (el costo por el préstamo)."

    # If asks about pagar extra
    m = re.search(r'pag(ar|o).*(extra|adicional|más)\s*(\d+\.?\d*)', q)
    if m:
        try:
            extra = float(m.group(3))
        except:
            extra = round(ultimo_monto * 0.05, 2) if ultimo_monto else 0.0
        if ultima_df is None:
            return "Genera la tabla primero para que calcule exactamente el efecto del pago extra."
        # simulate with simple loop
        tasa = ultimo_tasa; monto = ultimo_monto
        tasa_m = tasa/12/100
        saldo = monto
        cuota = ultima_df['Pago total'].iloc[0]
        pagos = 0
        intereses_acum = 0.0
        while saldo > 0.001 and pagos < 2000:
            interes = saldo * tasa_m
            pago_total = cuota + extra
            capital = max(0, pago_total - interes)
            saldo -= capital
            intereses_acum += interes
            pagos += 1
            if saldo <= 0:
                break
        ahorro = ultima_df['Interés'].sum() - intereses_acum
        return f"Si añades ${extra:,.2f} extra al mes, pagarías aproximadamente en {pagos} meses y ahorrarías ~${ahorro:,.2f} en intereses (estimación)."

    # If asks about refinancing
    if re.search(r'(refinanc|refinar|renegoc|cambiar.*tasa|tasa.*baj)', q):
        if ultimo_tasa is None:
            return "Genera la tabla primero para que pueda estimar el beneficio de refinanciar."
        objetivo = max(0.0, ultimo_tasa * 0.85)
        return f"Una meta razonable es intentar negociar por debajo de {objetivo:.2f}% anual. Cada punto porcentual menos reduce significativamente intereses."

    # If asks about plazo vs cuota
    if re.search(r'\b(plazo|cuota|meses|mensual)\b', q):
        return "Reducir el plazo baja los intereses totales pero aumenta la cuota mensual. Aumentar el plazo baja la cuota mensual pero aumenta intereses en total. Elige según flujo de caja."

    # If asks for total interests
    if re.search(r'(interes(es)?|costo total|total a pagar)', q):
        if ultima_df is None:
            return "Genera la tabla primero para ver los intereses totales."
        total_intereses = ultima_df['Interés'].sum()
        total_pagado = ultima_df['Pago total'].sum()
        return f"Intereses totales: ${total_intereses:,.2f}. Total pagado (capital+intereses): ${total_pagado:,.2f}."

    # If numeric question "cuanto ahorro"
    m2 = re.search(r'ahorr.*\s*(\d+\.?\d*)', q)
    if 'ahorro' in q and ultima_df is not None and m2:
        # assume asking saving from extra
        extra = float(m2.group(1))
        # reuse earlier logic
        tasa = ultimo_tasa; monto = ultimo_monto
        tasa_m = tasa/12/100
        saldo = monto
        cuota = ultima_df['Pago total'].iloc[0]
        pagos=0; intereses_acum=0.0
        while saldo>0.001 and pagos<2000:
            interes = saldo*tasa_m
            pago_total = cuota + extra
            capital = max(0, pago_total - interes)
            saldo -= capital
            intereses_acum += interes
            pagos+=1
            if saldo<=0: break
        ahorro = ultima_df['Interés'].sum() - intereses_acum
        return f"Con pago extra ${extra:,.2f} al mes, estimas ahorrar ~${ahorro:,.2f} y terminar en {pagos} meses."

    # fallback short domain-limited answer
    return ("Disculpa, puedo ayudar con dudas sobre esta tabla: amortización, pagos extra, plazo, tasa y refinanciamiento. "
            "Prueba: '¿Cuánto ahorro si pago extra 100?' o '¿conviene refinanciar a 6.5%?'")

# Optional OpenAI usage (if user provides OPENAI_API_KEY). Use carefully and only for finance domain.
def openai_finanbot_response(question):
    try:
        # Lazy import here to avoid hard dependency
        from openai import OpenAI
    except Exception as e:
        return None
    key = os.environ.get('OPENAI_API_KEY', None)
    if not key:
        return None
    # If openai available, call a short completions / chat
    try:
        client = OpenAI(api_key=key)
        prompt = (
            "Eres un asistente financiero conciso que solo responde preguntas sobre amortización, pagos extra, refinanciamiento, plazos y tasas.\n"
            f"Contexto: monto={ultimo_monto}, tasa={ultimo_tasa}, plazo={ultimo_plazo} meses.\n"
            f"Pregunta: {question}\n"
            "Responde en español, breve y con números si es necesario."
        )
        resp = client.responses.create(model="gpt-4o-mini", input=prompt, max_tokens=250)
        out = resp.output_text if hasattr(resp, 'output_text') else str(resp)
        return out
    except Exception as e:
        return None

# JS bridge: activar widget y enviar preguntas al backend
def activar_finanbot_widget():
    # show toggle and widget
    display(Javascript("""
      document.getElementById('finanbot_widget').style.display = 'block';
      document.getElementById('finanbot_toggle').style.display = 'none';
      document.getElementById('finanbot_status').innerText = 'listening';
      // ensure messages area shows a welcome
      const m = document.getElementById('finanbot_messages');
      m.innerHTML = "<div style='margin-bottom:8px;'><b>Finanbot:</b> Hola — soy Finanbot. Pregunta sobre la tabla (pagos extra, amortización, refinanciamiento).</div>";
      m.scrollTop = m.scrollHeight;
    """))

# Function called from JS to process question (registered as callback)
def procesar_pregunta_finanbot(texto):
    texto = str(texto)
    # If OpenAI is configured and permitted, try it first for a more natural answer
    if USE_OPENAI:
        try:
            out = openai_finanbot_response(texto)
            if out:
                return out
        except Exception:
            pass
    # fallback to local response
    return local_finanbot_response(texto)

# JS helper: define enviarPreguntaFinanbot function in the notebook frontend
display(Javascript("""
window.enviarPreguntaFinanbot = function(){
  const txt = document.getElementById('finanbot_text').value || '';
  if(!txt.trim()) return;
  const area = document.getElementById('finanbot_messages');
  area.innerHTML += "<div style='margin-bottom:8px;'><b>Tú:</b> " + txt.replace(/</g,'&lt;') + "</div>";
  area.scrollTop = area.scrollHeight;
  document.getElementById('finanbot_text').value = '';
  // call python backend
  google.colab.kernel.invokeFunction('procesar_pregunta_finanbot', [txt], {}).then((resp) => {
    // resp is an array-like from Colab; extract the result text
    let text = resp && resp[0] ? resp[0] : String(resp);
    area.innerHTML += "<div style='margin-bottom:8px; color:#0a2540;'><b>Finanbot:</b> " + text.replace(/</g,'&lt;') + "</div>";
    area.scrollTop = area.scrollHeight;
  });
};
"""))

# Register callback
output.register_callback('procesar_pregunta_finanbot', procesar_pregunta_finanbot)
output.register_callback('activar_finanbot_widget', activar_finanbot_widget)

# Also register a callback to close/hide Finanbot and show small toggle button
display(Javascript("""
document.getElementById('finanbot_toggle').onclick = function(){
  // show widget
  document.getElementById('finanbot_widget').style.display = 'block';
  this.style.display = 'none';
};
"""))

# ---------------------------
# === SIMULADOR (modal-like insert) ===
# ---------------------------
def abrir_simulador_modal():
    # inject a nice modal-like panel into resultado (keeps table visible)
    if ultima_df is None:
        display(Javascript("document.getElementById('resultado').innerHTML += '<div style=color:#b30000; margin-top:8px;>Calcula un crédito primero para usar el simulador.</div>';"))
        return
    js = r"""
    const cont = document.getElementById('resultado');
    // create panel
    cont.innerHTML += `
      <div id="sim_modal" style="margin-top:12px; padding:12px; border-radius:10px; background:linear-gradient(180deg,#fff,#f7fcff); border:1px solid #e6f4ff;">
        <h3 style="margin-top:0; color:#12355b;">🔁 Simulador de escenarios</h3>
        <div>
          <label>Tasa alternativa (%)</label>
          <input id="sim_tasa" type="number" placeholder="Ej. 6.5" />
          <label style="margin-top:8px;">Plazo alternativo (meses)</label>
          <input id="sim_plazo" type="number" placeholder="Ej. 24" />
        </div>
        <div style="margin-top:10px;">
          <button class="download-btn" onclick="google.colab.kernel.invokeFunction('procesar_simulador', [parseFloat(document.getElementById('sim_tasa').value)||0, parseInt(document.getElementById('sim_plazo').value)||0], {})">Comparar</button>
          <button class="download-btn" onclick="document.getElementById('sim_modal').remove()" style="margin-left:8px;">Cerrar</button>
        </div>
      </div>
    `;
    window.scrollTo(0, document.getElementById('sim_modal').offsetTop - 20);
    """
    display(Javascript(js))

def procesar_simulador(tasa_alt, plazo_alt):
    global ultima_df, ultimo_monto, ultima_tasa, ultimo_plazo
    if ultima_df is None:
        display(Javascript("document.getElementById('resultado').innerHTML += '<div style=color:#b30000; margin-top:8px;>Calcula un crédito primero.</div>';"))
        return
    try:
        tasa_alt = float(tasa_alt); plazo_alt = int(plazo_alt)
    except:
        display(Javascript("document.getElementById('resultado').innerHTML += '<div style=color:#b30000; margin-top:8px;>Ingresa valores válidos.</div>';"))
        return
    if tasa_alt<=0 or plazo_alt<=0:
        display(Javascript("document.getElementById('resultado').innerHTML += '<div style=color:#b30000; margin-top:8px;>Valores mayores a 0.</div>';"))
        return
    monto = ultimo_monto
    df_base = ultima_df
    df_alt = generar_tabla_amortizacion(monto, tasa_alt, plazo_alt)
    total_inter_base = df_base['Interés'].sum()
    total_inter_alt = df_alt['Interés'].sum()
    ahorro = total_inter_base - total_inter_alt
    # pretty verdict
    if ahorro > 0:
        sugerido = f"Te conviene el escenario alternativo: ahorras ${ahorro:,.2f} en intereses."
    elif ahorro < 0:
        sugerido = f"No conviene el escenario alternativo: pagarías ${-ahorro:,.2f} más en intereses."
    else:
        sugerido = "Ambos escenarios dan resultados similares."

    # grafico
    plt.figure(figsize=(4,2.2))
    bars = [total_inter_base, total_inter_alt]
    labels = ['Actual','Alternativa']
    x = np.arange(len(labels))
    plt.bar(x, bars)
    plt.xticks(x, labels)
    plt.ylabel('Intereses totales (USD)')
    plt.tight_layout()
    buf = io.BytesIO(); plt.savefig(buf, format='png', dpi=120, bbox_inches='tight'); plt.close(); buf.seek(0)
    b64 = base64.b64encode(buf.read()).decode('utf-8')

    html = f"""
      <div style="margin-top:12px; padding:12px; border-radius:10px; background:#f0f9ff; border:1px solid #dff0ff;">
        <h4 style="margin-top:0; color:#12355b;">Resultado</h4>
        <p><b>Intereses actuales:</b> ${total_inter_base:,.2f}</p>
        <p><b>Intereses alternativa:</b> ${total_inter_alt:,.2f}</p>
        <p><b>Ahorro estimado:</b> ${ahorro:,.2f}</p>
        <p style="font-weight:700;">{sugerido}</p>
        <p>Si quieres profundizar, pregunta al Finanbot en la esquina inferior derecha.</p>
        <div style="margin-top:8px;"><img src="data:image/png;base64,{b64}" style="width:100%; border-radius:8px;" /></div>
      </div>
    """
    display(Javascript(f"document.getElementById('resultado').innerHTML += `{html}`;"))

# ---------------------------
# === COMPONENTE EDUCATIVO (ANTES de generar tabla) ===
# ---------------------------
def abrir_componente_educativo_inicial():
    # This opens a "page" explaining basics — before table generation
    educ_html = r"""
    <div style="padding:12px; border-radius:12px; background:linear-gradient(180deg,#fff,#f8fbff);">
      <h2 style="color:#12355b; margin-top:0;">📚 Componente educativo</h2>
      <div style="background:#fff; padding:10px; border-radius:10px; border:1px solid #eef4fb;">
        <h3>¿Qué es un crédito? (explicado fácil)</h3>
        <p>Es pedir dinero para comprar algo ahora y devolverlo poco a poco. Piensa: pides una bici, y pagas mensualmente hasta completarla.</p>
        <svg width="100%" height="90" viewBox="0 0 200 90" xmlns="http://www.w3.org/2000/svg">
          <rect x="8" y="30" rx="10" ry="10" width="60" height="30" fill="#f3f8fc" stroke="#a7c4e0"/>
          <text x="12" y="50" fill="#12355b" font-size="12">Bici ahora →</text>
        </svg>
        <hr>
        <h4>Conceptos clave</h4>
        <ul>
          <li><b>Tasa de interés:</b> precio por pedir prestado.</li>
          <li><b>Plazo:</b> cuántos meses tardas en pagar.</li>
          <li><b>Cuota:</b> lo que pagas cada mes (capital + interés).</li>
        </ul>
        <div style="margin-top:8px;">
          <button class="download-btn" onclick="google.colab.kernel.invokeFunction('volver_inicio', [], {})">Volver</button>
        </div>
      </div>
    </div>
    """
    display(Javascript(f"document.getElementById('resultado').innerHTML = `{educ_html}`;"))

def volver_inicio():
    # re-render the original main container (we will simply clear resultado)
    display(Javascript("document.getElementById('resultado').innerHTML = '';"))

# ---------------------------
# === TASAS INTERNACIONALES (INTENTO EN VIVO) ===
# ---------------------------

# Note: algunas fuentes oficiales requieren API keys (Banxico, FRED, etc.). Aquí:
# - Si tienes variables de entorno con tokens (BANXICO_TOKEN, FRED_KEY o TRADING_ECONOMICS_KEY),
#   el código las usará automáticamente.
# - Si no hay tokens o la fuente bloquea peticiones, verás un mensaje y valores guía.
# Esto evita romper la UI y mantiene la apariencia intacta.

def safe_get(url, headers=None, timeout=9):
    try:
        r = requests.get(url, headers=headers or {}, timeout=timeout)
        if r.status_code == 200:
            return r.text
    except Exception:
        return None
    return None

def fetch_tasa_mexico():
    """
    Intentos (en orden):
    1) Banxico API si BANXICO_TOKEN = env var y se especifica el series id en BANXICO_SERIE (opcional).
    2) Scrape minimal de la página pública (intento, puede fallar).
    """
    token = os.environ.get('BANXICO_TOKEN', None)
    serie = os.environ.get('BANXICO_SERIE', None)  # optional: user can set the series code
    if token and serie:
        try:
            url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie}/datos/oportuno?token={token}"
            r = requests.get(url, timeout=8)
            if r.status_code == 200:
                js = r.json()
                val = js.get('bmx', {}).get('series', [])[0].get('datos', [])[0].get('dato', None)
                fecha = js.get('bmx', {}).get('series', [])[0].get('datos', [])[0].get('fecha', None)
                if val:
                    return f"{val} (fecha {fecha})"
        except Exception:
            pass
    # Fallback: try minimal scraping from Banxico home (best-effort; puede fallar si cambian la web)
    text = safe_get("https://www.banxico.org.mx")
    if text:
        m = re.search(r'(\d+\.\d+)\s*%.*?tasa.*?objetivo|tasa.*?intervención', text, re.IGNORECASE|re.DOTALL)
        if not m:
            m = re.search(r'(\d+\.\d+)\s*%.*?Banxico', text, re.IGNORECASE|re.DOTALL)
        if m:
            return f"{m.group(1)} % (extraído de banxico.org.mx — verificar)"
    return None

def fetch_tasa_colombia():
    """
    Intentos:
    - Scrape la página pública de Banco de la República para buscar 'Tasa de intervención'.
    - Si se requiere, permitir tokens mediante BANREP_TOKEN (no estándar aquí).
    """
    text = safe_get("https://www.banrep.gov.co")
    if text:
        m = re.search(r'Tasa de intervención.*?(\d+\.\d+)\s*%', text, re.IGNORECASE|re.DOTALL)
        if not m:
            m = re.search(r'(\d+\.\d+)\s*%.*?tasa.*?intervención', text, re.IGNORECASE|re.DOTALL)
        if m:
            return f"{m.group(1)} % (fecha estimada — extraído de banrep.gov.co)"
    return None

def fetch_tasa_eeuu():
    """
    Intentos:
    - Scrape la página de la Fed (resumen de política monetaria) buscando 'federal funds rate' o 'target range'.
    - If FRED_KEY provided you could query FRED; here we attempt scraping as best-effort.
    """
    text = safe_get("https://www.federalreserve.gov/monetarypolicy/openmarket.htm")
    if text:
        # Search for a percentage like "0.25 to 0.50 percent" or "1.50 percent"
        m = re.search(r'(\d+\.\d+)\s*(to\s*\d+\.\d+)?\s*percent', text, re.IGNORECASE)
        if m:
            # If a range detected, return both
            if m.group(2):
                rng = re.search(r'(\d+\.\d+)\s*to\s*(\d+\.\d+)\s*percent', text, re.IGNORECASE)
                if rng:
                    return f"{rng.group(1)}% – {rng.group(2)}% (target range — fuente Fed)"
            return f"{m.group(1)}% (Fed — extraído)"
    # fallback: try FRED (if key present)
    fred_key = os.environ.get('FRED_KEY', None)
    if fred_key:
        try:
            # The FRED series for effective federal funds rate is "EFFR"
            r = requests.get(f"https://api.stlouisfed.org/fred/series/observations?series_id=EFFR&api_key={fred_key}&file_type=json&limit=1")
            if r.status_code == 200:
                js = r.json()
                obs = js.get('observations', [])
                if obs:
                    val = obs[-1].get('value')
                    date = obs[-1].get('date')
                    if val and val != '.':
                        return f"{val}% (EFFR, fecha {date})"
        except Exception:
            pass
    return None

def fetch_tasa_brasil():
    # Try Banco Central do Brasil page
    text = safe_get("https://www.bcb.gov.br")
    if text:
        m = re.search(r'Selic.*?(\d+\.\d+)\s*%', text, re.IGNORECASE|re.DOTALL)
        if m:
            return f"{m.group(1)}% (Selic — extraído de bcb.gov.br)"
    return None

def fetch_tasa_espana():
    # For Spain / Euro area, use ECB website attempt
    text = safe_get("https://www.ecb.europa.eu/home/html/index.en.html")
    if text:
        m = re.search(r'(\d+\.\d+)\s*%.*?interest rate', text, re.IGNORECASE|re.DOTALL)
        if m:
            return f"{m.group(1)}% (ECB — extraído)"
    return None

# To keep behavior deterministic: if fetch functions return None, we show message and offer static fallback.
tasas_fallback = {
    "mexico": {"name":"México", "tasa":"(no disponible en automático; proporciona BANXICO_TOKEN y BANXICO_SERIE para datos exactos)"},
    "colombia": {"name":"Banco de la República (Colombia)", "tasa":"(no disponible en automático)"},
    "eeuu": {"name":"Federal Reserve (EE.UU.)", "tasa":"(no disponible en automático)"},
    "brasil": {"name":"Banco Central do Brasil", "tasa":"(no disponible en automático)"},
    "espana": {"name":"Banco Central Europeo / España", "tasa":"(no disponible en automático)"},
}

def abrir_tasas_inicial():
    # show initial menu before generating table
    menu_html = """
      <div style="margin-top:12px; padding:12px; border-radius:10px; background:#fff; border:1px solid #e8f1ff;">
        <h3 style="margin-top:0;color:#12355b;">🏦 Tasas bancarias internacionales (actuales)</h3>
        <p style="color:#4b6b8d;">Intentando obtener las tasas reales... si tu entorno no permite llamadas externas, verás una nota y valores guía.</p>
        <div style="display:flex; gap:8px; flex-wrap:wrap;">
          <button class="download-btn" onclick="google.colab.kernel.invokeFunction('mostrar_tasa_key_init', ['mexico'], {})">México</button>
          <button class="download-btn" onclick="google.colab.kernel.invokeFunction('mostrar_tasa_key_init', ['colombia'], {})">Colombia</button>
          <button class="download-btn" onclick="google.colab.kernel.invokeFunction('mostrar_tasa_key_init', ['eeuu'], {})">EE.UU.</button>
          <button class="download-btn" onclick="google.colab.kernel.invokeFunction('mostrar_tasa_key_init', ['brasil'], {})">Brasil</button>
          <button class="download-btn" onclick="google.colab.kernel.invokeFunction('mostrar_tasa_key_init', ['espana'], {})">España</button>
        </div>
      </div>
    """
    display(Javascript(f"document.getElementById('resultado').innerHTML = `{menu_html}`;"))

def mostrar_tasa_key_init(key):
    mapping = {
        "mexico": fetch_tasa_mexico,
        "colombia": fetch_tasa_colombia,
        "eeuu": fetch_tasa_eeuu,
        "brasil": fetch_tasa_brasil,
        "espana": fetch_tasa_espana,
    }
    fn = mapping.get(key)
    tasa = None
    if fn:
        try:
            tasa = fn()
        except Exception:
            tasa = None
    if tasa is None:
        d = tasas_fallback.get(key)
        html = f"""
          <div style="margin-top:12px; padding:12px; border-radius:10px; background:#f8fff6; border:1px solid #e8f6ea;">
            <h4 style="margin-top:0;color:#12355b;">{d['name']}</h4>
            <p><b>Tasa de referencia (actual):</b> {d['tasa']}</p>
            <p style="color:#4b6b8d;"><small>Nota: en este entorno Colab algunas APIs oficiales requieren tokens o bloquean scraping. Si quieres, proporciona las variables de entorno correspondientes (ej. BANXICO_TOKEN & BANXICO_SERIE, FRED_KEY) o dime y te añado un flujo con TradingEconomics o FRED específico.</small></p>
          </div>
        """
        display(Javascript(f"document.getElementById('resultado').innerHTML += `{html}`;"))
    else:
        html = f"""
          <div style="margin-top:12px; padding:12px; border-radius:10px; background:#f8fff6; border:1px solid #e8f6ea;">
            <h4 style="margin-top:0;color:#12355b;">{key.upper()}</h4>
            <p><b>Tasa de referencia (actual):</b> {tasa}</p>
          </div>
        """
        display(Javascript(f"document.getElementById('resultado').innerHTML += `{html}`;"))

# Second-page versions (same behavior)
def abrir_tasas_segunda():
    abrir_tasas_inicial()

# ---------------------------
# === COMPONENTE EDUCATIVO (SEGUNDA PÁGINA) ===
# ---------------------------
def abrir_componente_educativo_segunda():
    # More completo y cute (reemplaza #resultado)
    educ_html = r"""
      <div style="padding:12px; border-radius:12px; background:linear-gradient(180deg,#fff,#f8fbff);">
        <h2 style="color:#12355b; margin-top:0;">📚 Componente educativo — explicado fácil</h2>
        <div style="background:#fff; padding:10px; border-radius:10px; border:1px solid #eef4fb;">
          <h3>¿Qué es un crédito?</h3>
          <p>Es pedir dinero ahora y devolverlo poco a poco. Imagina que compras una bici y pagas mes a mes hasta que ya no debes nada.</p>
          <svg width="100%" height="80" viewBox="0 0 200 80" xmlns="http://www.w3.org/2000/svg">
            <circle cx="30" cy="50" r="12" fill="#ffe8e8" stroke="#ffc0c0"/>
            <rect x="60" y="28" rx="8" ry="8" width="110" height="36" fill="#eef6ff" stroke="#d7e7ff"/>
            <text x="66" y="50" fill="#12355b" font-size="12">Pides dinero → Pagas en cuotas</text>
          </svg>
          <hr>
          <h4>Más conceptos</h4>
          <ul>
            <li><b>Tasa de interés:</b> cuánto te cuesta pedir prestado.</li>
            <li><b>Plazo:</b> número de meses para pagar.</li>
            <li><b>Cuota:</b> pago mensual (capital + interés).</li>
          </ul>
          <h4>Tips fáciles</h4>
          <ol>
            <li>Paga extra si puedes: reduce tiempo y intereses.</li>
            <li>No uses más del 30-35% de tus ingresos en deudas.</li>
            <li>Compara tasas antes de firmar un crédito.</li>
          </ol>
          <div style="margin-top:8px;">
            <button class="download-btn" onclick="google.colab.kernel.invokeFunction('volver_a_tabla', [], {})">Volver a la tabla</button>
          </div>
        </div>
      </div>
    """
    display(Javascript(f"document.getElementById('resultado').innerHTML = `{educ_html}`;"))

# Volver a tabla
def volver_a_tabla():
    if ultima_df is not None:
        calc_credit_backend(ultimo_monto, ultima_tasa, ultimo_plazo)
    else:
        display(Javascript("document.getElementById('resultado').innerHTML = '<b>Calcula un crédito primero.</b>';"))

# ---------------------------
# === REGISTRO DE CALLBACKS ===
# ---------------------------
output.register_callback('calc_credit_backend', calc_credit_backend)
output.register_callback('descargar_excel', descargar_excel)
output.register_callback('abrir_dashboard', abrir_dashboard)
output.register_callback('activar_finanbot_widget', activar_finanbot_widget)
output.register_callback('procesar_pregunta_finanbot', procesar_pregunta_finanbot)
output.register_callback('abrir_simulador_modal', abrir_simulador_modal)
output.register_callback('procesar_simulador', procesar_simulador)
output.register_callback('abrir_componente_educativo_inicial', abrir_componente_educativo_inicial)
output.register_callback('abrir_tasas_inicial', abrir_tasas_inicial)
output.register_callback('mostrar_tasa_key_init', mostrar_tasa_key_init)
output.register_callback('abrir_componente_educativo_segunda', abrir_componente_educativo_segunda)
output.register_callback('abrir_tasas_segunda', abrir_tasas_segunda)
output.register_callback('volver_inicio', volver_inicio)
output.register_callback('volver_a_tabla', volver_a_tabla)

print("Listo — pega valores y prueba: 1) usa 'Integrar tasas internacionales' y 'Componente educativo' antes de calcular; 2) calcula para ver Dashboard → Finanbot (corner widget) → Simulador.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Listo — pega valores y prueba: 1) usa 'Integrar tasas internacionales' y 'Componente educativo' antes de calcular; 2) calcula para ver Dashboard → Finanbot (corner widget) → Simulador.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>